# SQL

**1. Цели исследования**<a id="step1"></a>

- Посчитать, сколько книг вышло после 1 января 2000 года
- Для каждой книги посчитать количество обзоров и среднюю оценку
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

**2. Исследовать таблицы — вывести первые строки**<a id="step2"></a>

---
Подготовим подключение к базе: импортируем библиотеки, установим параметры (имя пользователя, пароль, порт подключения, название базы данных) и сохраняем коннектор.

In [1]:
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': '', 
 'pwd': '', 
 'host': '',
 'port': , 
 'db': ''} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])


engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Далее выполним 5 SQL-запрос, сохраним их в соответствующие таблицы и выведем первые строки каждой.

In [2]:
book = ''' SELECT *             
            FROM books
            
        '''
books = pd.io.sql.read_sql(book, con = engine) 
display(books.head(5))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
author = ''' SELECT *             
            FROM authors
            
        '''
authors = pd.io.sql.read_sql(author, con = engine) 
display(authors.head(5))

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
publisher = ''' SELECT *             
            FROM publishers
            
        '''
publishers = pd.io.sql.read_sql(publisher, con = engine) 
display(publishers.head(5))

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
rating = ''' SELECT *             
            FROM ratings
            
        '''
ratings = pd.io.sql.read_sql(rating, con = engine) 
display(ratings.head(5))

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
review = ''' SELECT *             
            FROM reviews
            
        '''
reviews = pd.io.sql.read_sql(review, con = engine) 
display(reviews.head(5))

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**3. Сделать по одному SQL-запросу для решения каждого задания**<a id="step3"></a>

---
**Посчитать, сколько книг вышло после 1 января 2000 года**<a id="step6"></a>

---
Нужную выборку получаем оператором SELECT. В запросе два ключевых слова: SELECT и FROM.
SELECT выбирает нужные столбцы из таблицы базы данных. FROM (англ. «из») указывает, из какой таблицы брать данные. Начало условия, по которому отбираются данные, обозначают командой WHERE (англ. «где»). Проверка на соответствие условию проходит в каждой строке таблицы. Также потребуется функция COUNT(*) - возвращает общее количество строк в таблице.

Поэтому для выполнения этого запроса нам потребуется:
- в блок SELECT добавляем подсчет столбца book_id
- в блок FROM - базу данных, содержащую нужные сведения, т.е. books
- в блок WHERE - условие, т.е. дата публикации позже 01 января 2000 года. Поскольку нам нужны только те которые вышли позже, то мы не включаем вышедшие 1 января (а их в этот день выпустили 2 =)).

In [7]:
n_book = ''' SELECT 
            COUNT(book_id)
            
            FROM books
            
            WHERE
            publication_date > '2000-01-01'
        '''

In [8]:
n_books = pd.io.sql.read_sql(n_book, con = engine) 
display(n_books)

,count
0,819


Затем мы выполнили SQL запрос и сохранили результат в переменную n_books. После 1 января 2000 года было выпущено 819 книг.

**Для каждой книги посчитать количество обзоров и среднюю оценку**<a id="step7"></a>

---
Помимо рассмотренных в предыдущем пункте операторов нам потребуются еще:
- функция AVG (column) возвращает среднее значение по столбцу column;
- внешнее соединение таблиц методом LEFT JOIN - возьмём все данные левой таблицы и строки на пересечении левой и правой, которые удовлетворяют условию присоединения;
- команда GROUP BY (англ. «группировать по»), которую применяют, когда данные нужно разделить на группы по значениям полей.


Для выполнения этого запроса нам потребуется:
- в блок SELECT добавляем нужные нам столбцы - название книги title из таблицы books, средний рейтинг книги из таблицы ratings и количество отзывов из таблицы reviews
- в блок FROM - базу данных, на основе которой будет присоединение, т.е. books
- присоединяем таблицы методом LEFT JOIN по столбцу book_id, который присутствует во всех требуемых таблицах
- в блок GROUP BY добавим группировку по названию книги (title)

In [9]:
n_book_RR = ''' SELECT 
            books.title AS title,
            AVG(ratings.rating) AS avg_rating,
            COUNT(reviews.review_id) AS review_count
            
            FROM 
            books
            LEFT JOIN ratings ON ratings.book_id = books.book_id
            LEFT JOIN reviews ON reviews.book_id = books.book_id
            
            GROUP BY 
            title;

        '''

In [10]:
n_books_RR = pd.io.sql.read_sql(n_book_RR, con = engine) 
display(n_books_RR)

,title,avg_rating,review_count
0,The Count of Monte Cristo,4.217391,115
1,Count Zero (Sprawl #2),2.500000,4
2,The Botany of Desire: A Plant's-Eye View of th...,3.500000,4
3,The Poisonwood Bible,4.363636,110
4,The Canterbury Tales,3.333333,18
...,...,...,...
994,Of Love and Other Demons,4.500000,4
995,In the Heart of the Sea: The Tragedy of the Wh...,3.333333,9
996,Welcome to Temptation (Dempseys #1),5.000000,4
997,World's End (The Sandman #8),4.500000,4


Затем выполнили запрос и вывели на экран нужную таблицу, также сохранив ее в переменной n_books_RR.

**Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры**<a id="step8"></a>

---
Помимо рассмотренных в предыдущих пунктах операторов нам потребуются еще:
- ORDER BY, чтобы сортировать данные по указанным полям;
- DESC (от англ. descending, «нисходящий») сортирует данные по убыванию;
- команда LIMIT (англ. «предел») ограничивает количество строк в выводе. Её всегда указывают последней в запросе.


Для выполнения этого запроса нам потребуется:
- в блок SELECT добавляем нужные нам столбцы - название книги publisher из таблицы publishers и количество книг из таблицы books
- в блок FROM - базу данных, на основе которой будет присоединение, т.е. publishers
- присоединяем таблицы методом LEFT JOIN по столбцу publisher_id, который присутствует во всех требуемых таблицах
- в блок GROUP BY добавим группировку по названию издательства (publisher)
- в блок ORDER BY добавим упорядочить поубыванию (DESC)
- в LIMIT добавим 1, чтобы увидеть издательство с наибольшим количеством книг.

In [11]:
n_book_P = ''' SELECT 
            publishers.publisher AS publisher,
            COUNT(books.book_id) AS books_count
            
            FROM 
            publishers
            LEFT JOIN books ON publishers.publisher_id = books.publisher_id
            
            GROUP BY 
            publisher
            
            ORDER BY 
            books_count desc
            
            LIMIT
            1;

        '''

In [12]:
n_books_P = pd.io.sql.read_sql(n_book_P, con = engine) 
display(n_books_P)

,publisher,books_count
0,Penguin Books,42


Затем выполним запрос и выведем на экран нужную таблицу, также сохранив ее в переменной n_books_P. Издательство с наибольшим количеством книг - Penguin Books, которое выпустило 42 книги.

**Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками**<a id="step9"></a>

---
Помимо рассмотренных в предыдущих пунктах операторов нам потребуются еще:
- конструкция HAVING (англ. «обладающий») — аналог WHERE для агрегированных функций
- подзапрос, т.е. запрос в запросе. Подзапросы могут выполняться в разных частях запроса. Если подзапрос записать в блоке FROM, то SELECT выберет данные из таблицы, полученной в результате работы подзапроса. Имя этой таблицы указывают во внутреннем запросе, к её столбцам обращаются во внешнем. Подзапрос записывают в круглых скобках.


Для выполнения этого запроса нам потребуется:
- в блок SELECT добавляем нужные нам столбцы - автора и средний рейтинг книги из подзапроса SUBQ
- в блок FROM - подзапрос SUBQ, в котором:
   - в блок SELECT добавляем нужные нам столбцы - автора из таблицы авторов и средний рейтинг книги из таблицы рейтинга и название книг из таблицы books
   - в блок FROM базу данных, на основе которой будет присоединение, т.е. authors. Присоединяем таблицы методом LEFT JOIN books и authors - по столбцу author_id, books и ratings - по book_id
   - в блок GROUP BY добавим группировку по авторуи названию книги
   - в блок HAVING условие, что оценок должно быть больше 50
   - в блок ORDER BY добавим упорядочить по убыванию (DESC) столбца среднего рейтинга книги

- в блок GROUP BY добавим группировку по автору
- в блок ORDER BY добавим упорядочить по убыванию (DESC) среднего рейтинга книг автора
- в LIMIT добавим 1, чтобы увидеть автора с самой высокой средней оценкой книг.

In [13]:
max_author = ''' SELECT 
            SUBQ.author AS author,
            AVG(SUBQ.avg_rating) as avg
            
            FROM 
            (SELECT 
            authors.author AS author,
            AVG(ratings.rating) AS avg_rating,
            books.title AS title
            
            FROM
            authors
            LEFT JOIN books ON authors.author_id = books.author_id
            LEFT JOIN ratings ON books.book_id = ratings.book_id
            
            GROUP BY 
            author,
            title
            
            HAVING 
            COUNT(ratings.rating) > 50 
            
            ORDER BY 
            avg_rating DESC) AS SUBQ
            
            GROUP BY 
            author
            
            ORDER BY 
            avg DESC
            
            LIMIT
            1;
        '''

In [14]:
max_authors = pd.io.sql.read_sql(max_author, con = engine) 
display(max_authors)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Затем выполним запрос и выведем на экран нужного автора, также сохранив его в переменной max_authors. Автор с самой высокой средней оценкой книг - Джоан Роулинг и ее вселенная магии и волшебства=).

**Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок**<a id="step10"></a>

---
Все необходимые функции и операторы есть в предыдущих запросах.


Для выполнения этого запроса нам потребуется запрос с подзапросом и еще одним подзапросом в нем:
- в блок SELECT добавляем среднее значение количества обзоров от пользователей, которые поставили больше 50 оценок, т.е. искомая величина
- в блок FROM - подзапрос SUBQ1, в котором:
   - в блок SELECT добавляем нужные нам столбцы - юзера из подзапроса SUBQ и количество обзоров от пользователей, которые поставили больше 50 оценок
   - в блок FROM добавим LEFT JOIN по столбцу username таблиц reviews и подзапроса SUBQ, в котором:
       - в блок SELECT уникальное имя пользователя и подсчет оценок, выставленных им
       - в блок FROM - таблицу, т.е. ratings
       - в блок GROUP BY добавим группировку по юзеру
       - в блок HAVING условие, что оценок должно быть больше 50
   - в блок GROUP BY добавим группировку по юзеру из подзапроса SUBQ
   - в блок ORDER BY добавим упорядочить по убыванию (DESC) количества обзоров от пользователей, которые поставили больше 50 оценок.

In [15]:
avg_user = ''' SELECT
            AVG(SUBQ1.cnt)
            
            FROM
            (SELECT
            SUBQ.username AS user,
            COUNT(reviews.review_id) as cnt
            
            FROM 
            
            (SELECT
            DISTINCT(username),
            COUNT(rating_id)
            
            FROM
            ratings
            
            GROUP BY 
            username
            
            HAVING 
            COUNT(rating_id) > 50) AS SUBQ
            
            LEFT JOIN reviews ON SUBQ.username = reviews.username
            
            GROUP BY 
            SUBQ.username
            
            ORDER BY 
            cnt DESC) AS SUBQ1;
        '''

In [16]:
avg_users = pd.io.sql.read_sql(avg_user, con = engine) 
display(avg_users)

,avg
0,24.333333


Затем выполним запрос и выведем на экран требуемую величину, также сохранив его в переменной avg_users. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 обзора.

Соберем данные, полученные в нашем исследовании:
1. После 1 января 2000 года было выпущено 819 книг.
2. Издательство с наибольшим количеством книг - Penguin Books, которое выпустило 42 книги.
3. Автор с самой высокой средней оценкой книг - Джоан Роулинг и ее вселенная магии и волшебства.
4. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 обзора.